In [1]:
import collections
import itertools  
import matplotlib.pyplot as plt
import networkx as nx
import os
import tellurium as te
import tesbml
import os, sys
cwd = os.getcwd()
print(cwd)

/Users/woosubshin/Desktop/ModelEngineering/SBMLLint/SBMLLint/notebook


In [2]:
#parent_dir = os.path.abspath(os.path.join(cwd, os.pardir, 'SBMLLint'))
parent_dir = os.path.abspath(os.path.join(cwd, os.pardir))
print(parent_dir)

/Users/woosubshin/Desktop/ModelEngineering/SBMLLint/SBMLLint


In [3]:
os.chdir(parent_dir)
print(os.getcwd())

/Users/woosubshin/Desktop/ModelEngineering/SBMLLint/SBMLLint


In [4]:
#import os, sys, inspect
#import inspect
#cmd_folder = os.path.realpath(os.path.abspath(os.path.split(inspect.getfile( inspect.currentframe() ))[0]))
#if cmd_folder not in sys.path:
#     sys.path.insert(0, cmd_folder)

In [5]:
from SBMLLint.common import constants as cn
from SBMLLint.common.molecule import Molecule
from SBMLLint.common.reaction import Reaction
from SBMLLint.games import print_model as pm

In [6]:
os.getcwd()

'/Users/woosubshin/Desktop/ModelEngineering/SBMLLint/SBMLLint'

In [7]:
print(cwd)

/Users/woosubshin/Desktop/ModelEngineering/SBMLLint/SBMLLint/notebook


In [8]:
num = 17
format_num = format(num, '03d')
document = tesbml.readSBML(os.path.join(os.getcwd(), 'SBMLLint/games/data/curated_' + format_num + '.xml'))
model = document.getModel()
pm.print_model(model)

<Model Hoefnagel2002_PyruvateBranches "Hoefnagel2002_PyruvateBranches">
R1: ADP + NAD + halfglucose -> ATP + NADH + pyruvate;
R2: NADH + pyruvate -> NAD + lactate;
R3: NAD + pyruvate + CoA -> NADH + AcCoA;
R4: AcCoA + PO4 -> CoA + AcP;
R5: ADP + AcP -> ATP + Ac;
R6: NADH + AcCoA -> NAD + CoA + AcO;
R7: NADH + AcO -> NAD + EtOH;
R8: pyruvate -> AcLac;
R9: AcLac -> AcetoinIn;
R10: AcetoinIn -> AcetoinOut;
R11: NADH + AcetoinIn -> NAD + Butanediol;
R12: ATP -> ADP;
R13: NADH + O2 -> NAD;
R14: AcLac -> AcetoinIn;


In [9]:
#ReactionCategory = collections.namedtuple('ReactionCategory',
#    'category predicate')
#MoleculeStoichiometry = collections.namedtuple('Molecule', 'molecule stoichiometry')
#MoleculeStoichiometry(molecule = Molecule.molecules[0], stoichiometry = 2)

In [10]:
"""Set of Molecules(SOM)"""

from SBMLLint.common import constants as cn
from SBMLLint.common.molecule import Molecule
from SBMLLint.common.reaction import Reaction
from SBMLLint.common.simple_sbml import SimpleSBML

import numpy as np

In [100]:
BRACKET_OPEN = "{"
BRACKET_CLOSE = "}"

class SOM(object):
    soms = []  # All SOMs. 
    def __init__(self, molecule):
        """
        :param molecules Molecule instances:
        :param reactions Reaction instances:
        """
        self.molecules = {molecule}
        self.reactions = set()
        self.identifier = self.makeId()
        self.__class__.addSOM(self)

    def __repr__(self):
        return self.identifier        
        
    def makeId(self):
        """
        Creates an identifier for the SOM to uniquely
        identifies its elements.
        :return str:
        """
        def joinMoleculeNames(molecules):
          names = [m.name for m in molecules]
          names.sort()
          return ', '.join(names)
        #
        identifier = "%s%s%s" % (
            BRACKET_OPEN, 
            joinMoleculeNames(list(self.molecules)),
            BRACKET_CLOSE
            )
        return identifier
        
    @classmethod    
    def addSOM(cls, new_som):
        if any([new_som.molecules.intersection(s.molecules) for s in cls.soms]):
          pass
        else:
          cls.soms.append(new_som)
    
    @classmethod
    def findSOM(cls, molecule):
        """
        Find the SOM that contains molecule
        and returns SOM
        :param Molecule molecule
        :return SOM
        """    
        for som in cls.soms:
            #if molecule in som.molecules:
            #    return som
            for m in som.molecules:
                if molecule.name == m.name:
                    return som
            
    @classmethod
    def merge(cls, reaction):
        """
        Merges two SOMs using a UniUni reaction 
        and updates cls.soms
        :param Reaction reaction
        :return SOM merged som
        """ 
        if reaction.category != cn.REACTION_1_1:
            raise AttributeError("This reaction cannot merge. You need a 1-1 reacton")
            pass
        som1 = cls.findSOM(reaction.reactants[0].molecule)
        som2 = cls.findSOM(reaction.products[0].molecule)
        print("som1 is: ", som1)
        print("som2 is: ", som2)
        
        if som1 == som2:
            print("Reactant and product are in the same som!")
            pass
        else: 
            som1.molecules = som1.molecules.union(som2.molecules)
            som1.reactions.add(reaction)
            som1.identifier = som1.makeId()
            cls.soms.remove(som2)
            return som1
        
    @classmethod
    def reduce(cls, reaction):
        """
        Reduces reaction using existing cls.soms
        param Reaction reaction
        return reduced reaction if reduced
        return False if reaction is not reducible
        """
        

In [ ]:
# Test SOM.reduce using model #17. Use SOM obtained by reaction12 to reduce 1

In [114]:
sbml_reaction1 = model.getReaction(0)
sbml_reaction2 = model.getReaction(11)

In [115]:
Reaction(sbml_reaction1)
Reaction(sbml_reaction2)
Reaction.reactions

[ADP + NAD + halfglucose -> ATP + NADH + pyruvate, ATP -> ADP]

In [116]:
print(Reaction.reactions[1].reactants)
print(Reaction.reactions[1].products)

[MoleculeStoichiometry(molecule=ATP, stoichiometry=1.0)]
[MoleculeStoichiometry(molecule=ADP, stoichiometry=1.0)]


In [117]:
print(Molecule.molecules)

[ADP, NAD, halfglucose, ATP, NADH, pyruvate]


In [118]:
for mol in Molecule.molecules:
    SOM(mol)
print(SOM.soms)

[{NAD}, {halfglucose}, {ADP, ATP}, {NADH}, {pyruvate}]


In [119]:
reaction1 = Reaction.reactions[0]
reaction2 = Reaction.reactions[1]
SOM.merge(reaction)

som1 is:  {ADP, ATP}
som2 is:  {ADP, ATP}
Reactant and product are in the same som!


In [121]:
SOM.soms

[{NAD}, {halfglucose}, {ADP, ATP}, {NADH}, {pyruvate}]

In [123]:
print(reaction1.reactants)
print(reaction1.products)

[MoleculeStoichiometry(molecule=ADP, stoichiometry=1.0), MoleculeStoichiometry(molecule=NAD, stoichiometry=1.0), MoleculeStoichiometry(molecule=halfglucose, stoichiometry=1.0)]
[MoleculeStoichiometry(molecule=ATP, stoichiometry=1.0), MoleculeStoichiometry(molecule=NADH, stoichiometry=1.0), MoleculeStoichiometry(molecule=pyruvate, stoichiometry=1.0)]


In [126]:
reactants = [ms.molecule for ms in reaction1.reactants]
products = [ms.molecule for ms in reaction1.products]
print(reactants)
print(products)

[ADP, NAD, halfglucose]
[ATP, NADH, pyruvate]


In [130]:
print(SOM.soms)
som1 = SOM.soms[0]
som2 = SOM.soms[2]
som2

[{NAD}, {halfglucose}, {ADP, ATP}, {NADH}, {pyruvate}]


{ADP, ATP}

In [ ]:
subset_rct = set(reactants).intersection

In [133]:
    def reduce(self, som):
        rct_subset = set(self.reactants).intersection(som)
        pdt_subset = set(self.products).intersection(som)
        rct_index = 0
        pdt_index = 0
        
        # Return False if the reaction is not reducible 
        if (not bool(rct_subset)) | (not bool(pdt_subset)):
            return False
        
        while bool(rct_subset) & bool(pdt_subset):
            rct = self.reactants[rct_index]
            pdt = self.products[pdt_index]

            # find reactant and product within given som
            while (rct not in rct_subset):
                rct_index = rct_index + 1
                rct = self.reactants[rct_index]
            while (pdt not in pdt_subset):
                pdt_index = pdt_index + 1
                pdt = self.products[pdt_index]

            # subtract stoichiometry from each side
            self.stoichiometry[0][rct_index] = self.stoichiometry[0][rct_index] - 1
            self.stoichiometry[1][pdt_index] = self.stoichiometry[1][pdt_index] - 1

            # remove element if stoichiometry becomes zero
            if self.stoichiometry[0][rct_index] == 0:
                rct_subset.remove(rct)
                del self.reactants[rct_index]
                del self.stoichiometry[0][rct_index]

            if self.stoichiometry[1][pdt_index] == 0:
                pdt_subset.remove(pdt)
                del self.products[pdt_index]
                del self.stoichiometry[1][pdt_index]
        
        # Update category because the reaction was reduced
        self.updateCategory()
        
        return True
  
    def updateCategory(self):
        
        # if either side is empty, we set None for the category and stoichiometry
        if (len(self.reactants)==0) | (len(self.products)==0) | \
           (self.reactants==[cn.EMPTYSET]) | (self.products==[cn.EMPTYSET]):
            self.category = None
        elif (len(self.reactants)==1) & (len(self.products)==1):
            self.category = cn.UNIUNI
        elif (len(self.reactants)==1) & (len(self.products)!=1):
            self.category = cn.UNIMULTI
        elif (len(self.reactants)!=1) & (len(self.products)==1):
            self.category = cn.MULTIUNI
        else:
            self.category = cn.MULTIMULTI
        print("Checking category...")

In [39]:
Reaction.reactions[0]._getCategory()

'reaction_n_1'